In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

plt.style.use('ggplot')

import nltk

In [ ]:
import csv
df = pd.read_csv('/content/Reviews.csv', on_bad_lines='skip', quoting=csv.QUOTE_NONE)
df.head()

In [ ]:
df['Text'].values[1]

In [ ]:
df = df.head(500)
print(df.shape)

In [ ]:
ax = df['Score'].value_counts().sort_index().plot(kind = 'bar', title = 'Count fo reveiws by stars', figsize = (10,5))
ax.set_xlabel('Review Stars')
plt.show()

In [ ]:
example = df['Text'].values[50]
print(example)

In [ ]:
nltk.download('punkt')
tokens = nltk.word_tokenize(example)
tokens[:10]

part of speech

In [ ]:
nltk.download('averaged_perceptron_tagger')
tagged = nltk.pos_tag(tokens)
tagged[:10]

group the tokens into chunks of text
pprint = pretty print

In [ ]:
nltk.download('maxent_ne_chunker')
nltk.download('words')
entities = nltk.chunk.ne_chunk(tagged)
entities.pprint()

Vader sentiment scoring

contains words with pos, neg, neu scoring

In [ ]:
nltk.download('vader_lexicon')
from nltk.sentiment import SentimentIntensityAnalyzer
from tqdm.notebook import tqdm

sia = SentimentIntensityAnalyzer()

In [ ]:
sia.polarity_scores("I am so happy!")

0 negative, neutral 0.3, positive 0.6, compound = negative to positive

In [ ]:
sia.polarity_scores('This is the worst thing ever')

mostly negatvie and neutral, compund score si more on the negative side

In [ ]:
sia.polarity_scores(example)

no polarity score

In [ ]:
df

TQDM = PROGRESS BAR FOR processing data

In [ ]:
res= {}
for i, row in tqdm(df.iterrows(), total = len(df)):
  text = row['Text']
  myid = row['Id']
  res[myid] = sia.polarity_scores(text)
  break

.T flips everything horizontally

In [ ]:
pd.DataFrame(res).T

 Vader model

In [ ]:
vaders = pd.DataFrame(res).T
vaders = vaders.reset_index().rename(columns= {'index': 'Id'})
vaders = vaders.merge(df, how = 'left')

In [ ]:
vaders.head()

In [ ]:
ax = sns.barplot(data = vaders, x = 'Score', y = 'vader_compound')
ax.set_title('Compound ScOre by amazons tar review')
plt.show()

In [ ]:
fig, axs = plt.subplots(1, 3, figsize = (12,3))
sns.barplot(data = vaders, x='Score', y = 'vader_pos', ax = axs[0])
sns.barplot(data = vaders, x='Score', y = 'vader_neu', ax = axs[1])
sns.barplot(data = vaders, x='Score', y = 'vader_neg', ax = axs[2])
axs[0].set_title('Positive')
axs[1].set_title('Neutral')
axs[2].set_title('Negative')
plt.show()

  Roberta pretrained model

In [ ]:
from transformers import AutoTokenizer
from transformers import AutoModelForSequenceClassification
from scipy.special import softmax

In [ ]:
MODEL = f"cardiffnlp/twitter-roberta-base-sentiment"
tokenizer = AutoTokenizer.from_pretrained(MODEL)
model = AutoModelForSequenceClassification.from_pretrained(MODEL)

In [ ]:
print(example)
sia.polarity_scores(example)

pt = pytorch

In [ ]:
encoded_text = tokenizer(example, return_tensors = 'pt')
output = model(**encoded_text)
scores = output[0][0].detach().numpy()
scores = softmax(scores)
scores_dict = {
    'roberta_neg' : scores[0],
    'roberta_neu' : scores[1],
    'roberta_pos' : scores[2]

}
print(scores_dict)

much more specific scores

In [ ]:
def polarity_scores_roberta(example):
  encoded_text = tokenizer(example, return_tensors = 'pt')
  output = model(**encoded_text)
  scores = output[0][0].detach().numpy()
  scores = softmax(scores)
  scores_dict = {
      'roberta_neg' : scores[0],
      'roberta_neu' : scores[1],
      'roberta_pos' : scores[2]
      }
  return scores_dict

In [ ]:
res = {}
for i, row in tqdm(df.iterrows(), total = len(df)):
  try:
    text = row['Text']
    myid = row['Id']
    vader_result = sia.polarity_scores(text)
    vader_result_rename = {}
    for key, value in vader_result.items():
      vader_result_rename[f"vader_{key}"] = value

    roberta_result = polarity_scores_roberta(text)
    both = {**vader_result_rename, **roberta_result}
    res[myid] = both
  except RunTimeError:
    print(f'Broke for id {myid}')

In [ ]:
results_df = pd.DataFrame(res).T
results_df = results_df.reset_index().rename(columns= {'index': 'Id'})
results_df = results_df.merge(df, how = 'left')

In [ ]:
results_df.head()

In [ ]:
results_df.columns

In [ ]:
sns.pairplot(data = results_df, vars=['vader_neg', 'vader_neu', 'vader_pos', 'roberta_neg', 'roberta_neu', 'roberta_pos'],
             hue = 'Score',
             palette = 'tab10')
plt.show()

  Text that the model judges as positive but is actually given a ratin of 1 star

In [ ]:
results_df.query('Score == 1').sort_values('roberta_pos', ascending=False)['Text'].values[0]

In [ ]:
results_df.query('Score == 1').sort_values('vader_pos', ascending=False)['Text'].values[0]

Positive review classified as negative

In [ ]:
results_df.query('Score == 5').sort_values('roberta_neg', ascending=False)['Text'].values[0]

In [ ]:
results_df.query('Score == 5').sort_values('vader_neg', ascending=False)['Text'].values[0]

In [ ]:
from transformers import pipeline

sent_pipeline = pipeline("sentiment-analysis")

In [ ]:
sent_pipeline('I love sentiment analysis')

In [ ]:
sent_pipeline("I don't think this is gonna work out")

In [ ]:
sent_pipeline("I guess it's best for both of us")

In [ ]:
sent_pipeline("Do I look like I'm happy")